In [1]:
import os
os.chdir('..')

In [2]:
from fox.config import Config
from fox.yolov3.utils.datasets import LoadImagesAndLabels
from fox.utils import parse_data_cfg
from fox.dataset import ComboDataset
import torch

In [3]:
c = Config(USE_PLANERCNN=False, BATCH_SIZE=2, EPOCHS=4)
combo_dataset = ComboDataset(c)
combo_loader = torch.utils.data.DataLoader(combo_dataset, batch_size=2, shuffle=True, collate_fn=combo_dataset.collate_fn)

def collate_fn(batch):
    yolo_data = LoadImagesAndLabels.collate_fn(batch)
    return yolo_data[0].float() / 255, None, yolo_data, None

data = parse_data_cfg("data/mini/yolo/custom.data")
yolo_dataset = LoadImagesAndLabels(
    data["train"],
    c.yolo_config.opt.img_size[1],
    c.BATCH_SIZE,
    augment=True,
    hyp=c.yolo_config.hyp,
    rect=c.yolo_config.opt.rect,
    cache_images=c.yolo_config.opt.cache_images,
    single_cls=False,
    mosiac=False,
    label_files_path=data["labels"]
)
yolo_loader = torch.utils.data.DataLoader(yolo_dataset, batch_size=c.BATCH_SIZE, shuffle=True, pin_memory=True, collate_fn=collate_fn)

Caching labels data/mini/yolo/train.txt (64 found, 0 missing, 0 empty, 0 duplicate, for 64 images): 100%|██████████| 64/64 [00:00<00:00, 1640.42it/s]
Caching labels data/mini/yolo/train.txt (64 found, 0 missing, 0 empty, 0 duplicate, for 64 images): 100%|██████████| 64/64 [00:00<00:00, 1254.77it/s]


In [7]:
%%timeit -n 1 -r 1
for i, _ in enumerate(combo_loader):
    if i == 3:
        break

2.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
%%timeit -n 1 -r 1
for i, _ in enumerate(yolo_loader):
    if i == 3:
        break

219 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
import numpy as np

In [7]:
a = np.random.randn(480, 640)
a.shape

(480, 640)

In [8]:
m = a < 0.5

In [16]:
%timeit m.sum()

282 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%timeit m.max() == True

5.9 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
